# **Hercoderen gemeenten 2022**

In 2022 zijn er een aantal gemeenten samengevoegd tot nieuwe gemeenten. Dit werkt door op een aantal plaatsen in de database:

2. In de data voor de Rioolwaterzuiveringsinstallaties (RWZI's). Waarschijnlijk wordt dit gewijzigd aangeleverd door RIVM, maar voor de zekerheid is er code ontwikkeld om dit op te vangen mocht de data in de oude indeling worden aangeleverd.



# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb"
    ]
}
```

# **TABLES**

---


### STATICS

Om de gemeentelijke indeling te kunnen aanpassen naar de gewenste indeling wordt er een mapping tabel gemaakt die de wijzigingen bevat. Met behulp van deze mapping-tabel wordt de data die van VWSTAGE naar VWSSTATIC wordt gekopieerd, direct in de goede gemeentelijke indeling gezet. Dit gebeurt in de Stored Procedure `SP_CBS_POPULATION_BASE_STATIC`.

N.B. de kolom "Active" in de tabel `Recode_Municipality` staat default op 0, wat betekent dat deze grenswijzigingen _NIET_ worden uitgevoerd. Om ze actief te maken moet met een update commando de waarde in de kolom "Active" op 1 worden gezet: 

```sql
Update [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
SET [ACTIVE] = 1
FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING]
WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])
```

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) CREATE STATIC TABLE(S).....
DROP TABLE IF EXISTS [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING];
GO

-- THIS TABLE CONTAINS THE REQUIRED MAPPING REGARDING THE RECLASSIFICATIONS OF MULTIPLE MUNCIPALITIES.
CREATE TABLE [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] (
	[ID] [INT] PRIMARY KEY IDENTITY (1,1)
   ,DATE_LAST_INSERTED [DATETIME] NOT NULL DEFAULT GETDATE()
   ,[GM_CODE_ORIGINAL] [NVARCHAR](10)
   ,[GM_NAME_ORIGINAL] [NVARCHAR](255)
   ,[GM_CODE_NEW] [NVARCHAR](10)
   ,[GM_NAME_NEW] [NVARCHAR](255)
   ,[VR_CODE] [NVARCHAR](10)
   ,[VR_NAME] [NVARCHAR](255)
   ,[PROVINCE_CODE] [NVARCHAR](10)
   ,[PROVINCE_NAME] [NVARCHAR](255)
   ,[GGD_CODE] [NVARCHAR](10)
   ,[GGD_NAME] [NVARCHAR](255)
   ,[SHARES] [DECIMAL](16,6) NOT NULL DEFAULT (1.0)
   ,[ACTIVE] [INT] NOT NULL DEFAULT(0)
) ON [PRIMARY];
GO

INSERT INTO [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] (
   [GM_CODE_ORIGINAL],
   [GM_NAME_ORIGINAL],
   [GM_CODE_NEW],
   [GM_NAME_NEW],
   [VR_CODE],
   [VR_NAME],
   [PROVINCE_CODE],
   [PROVINCE_NAME],
   [GGD_CODE],
   [GGD_NAME],
   [SHARES],
   [ACTIVE]) -- MUST BE SET TO 1 TO ALLOW PROPER MAPPING
VALUES 
   ('GM1684','Cuijk','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
   ,('GM0756','Boxmeer','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	,('GM0815','Mill en Sint Hubert','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	,('GM1702','Sint Anthonis','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	,('GM0786','Grave','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	,('GM1685','Landerd','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	,('GM0856','Uden','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	,('GM0398','Heerhugowaard','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,1)
	,('GM0416','Langedijk','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,1)
	,('GM0370','Beemster','GM0439','Purmerend','VR11','Zaanstreek-Waterland','PV27','Noord-Holland','GG7306','GGD Zaanstreek/Waterland',1,1)
   ,('GM0457','Weesp','GM0363','Amsterdam','VR13','Amsterdam-Amstelland','PV27','Noord-Holland','GG3406','GGD Amsterdam',1,1);
GO

# **OTHER**

---


Hier wordt gebruik gemaakt van de al aanwezige stored procedure `SP_MERGE_MOVE_STATIC_CBS_RWZI` om wijzigingen aan te brengen in de mapping van RWZI's naar gemeenten. Hiervoor wordt er een aantal regels toegevoegd aan het csv-bestand `Aantal_inwoners_per_verzorgingsgebied_mutaties**.csv` in de blob-storage, `datafiles\STATIC` map.